In [24]:
import pandas as pd
import plotly as py
import plotly.graph_objects as go
import math

In [310]:
def addEdgemap(start, end, edge_x, edge_y, lengthFrac=1, arrowPos = None, arrowLength=0.025, arrowAngle = 30, dotSize=20):
    
    # Get start and end cartesian coordinates
    x0, y0 = start
    x1, y1 = end
    
    # Incorporate the fraction of this segment covered by a dot into total reduction
    length = math.sqrt( (x1-x0)**2 + (y1-y0)**2 )
    dotSizeConversion = .0565/20 # length units per dot size
    convertedDotDiameter = dotSize * dotSizeConversion
    lengthFracReduction = convertedDotDiameter / length
    lengthFrac = lengthFrac - lengthFracReduction

    # Append line corresponding to the edge
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None) # Prevents a line being drawn from end of this edge to start of next edge
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)
    
    
    if not arrowPos == None:
        
        # Find the point of the arrow; assume is at end unless told middle
        pointx = x1
        pointy = y1
        eta = math.degrees(math.atan((x1-x0)/(y1-y0)))
        
        if arrowPos == 'middle' or arrowPos == 'mid':
            pointx = x0 + (x1-x0)/2
            pointy = y0 + (y1-y0)/2
            
        # Find the directions the arrows are pointing
        signx = (x1-x0)/abs(x1-x0)
        signy = (y1-y0)/abs(y1-y0)
        
        # Append first arrowhead
        dx = arrowLength * math.sin(math.radians(eta + arrowAngle))
        dy = arrowLength * math.cos(math.radians(eta + arrowAngle))    
        edge_x.append(pointx)
        edge_x.append(pointx - signx**2 * signy * dx)
        edge_x.append(None)
        edge_y.append(pointy)
        edge_y.append(pointy - signx**2 * signy * dy)
        edge_y.append(None)
        
#         # And second arrowhead
        dx2 = arrowLength * math.sin(math.radians(eta - arrowAngle))
        dy2 = arrowLength * math.cos(math.radians(eta - arrowAngle))    
        edge_x.append(pointx)
        edge_x.append(pointx - signx**2 * signy * dx2)
#         edge_x.append(None)
        edge_y.append(pointy)
        edge_y.append(pointy - signx**2 * signy * dy2)
#         edge_y.append(None)
        
#         edge_x.append(pointx - signx**2 * signy * dx)
#         edge_y.append(pointy - signx**2 * signy * dy)
        edge_x.append(None)
        edge_y.append(None)
        

    
    return edge_x, edge_y

In [265]:
towers=pd.read_csv('../data/towers_min.csv',index_col=0)
calls=pd.read_csv('../data/data.csv')

In [266]:
df=pd.merge(calls,towers,on='TowerID')

In [267]:

people=dict(type='scattermapbox',lat=df['lat'],lon=df['lon'],mode='markers')
fig=go.Figure(people,layout={
    'mapbox_style':'open-street-map',
  
    'mapbox':dict(
     
        bearing=0,
        center=dict(
            lat=23.2599,
            lon=77.4126
        ),
        pitch=0,
        zoom=10
    )
    
})

In [268]:
fig.show()

In [269]:
df['Date']=pd.to_datetime(df['Date'],format='%d-%m-%Y')
f=df[df['Date'].isin(pd.date_range('2020/06/05','2020/06/10'))].sort_values(['Date','Time'])


In [270]:
f

,Caller,Receiver,Date,Time,Duration,TowerID,IMEI,radio,lon,lat,range
2,9109709886,7694823496,2020-06-06,02:51:03,23,40478-38009-10112,60701793772294,GSM,77.422714,23.184586,1000
88,9109709886,7217467582,2020-06-06,16:13:35,33,40493-1150-38981,910312854667165,GSM,77.400742,23.268013,1000
135,9109709886,7340496772,2020-06-07,01:44:39,36,40458-2131-13202,446206666152640,GSM,77.402802,23.231277,1000
41,9109709886,9217826663,2020-06-10,14:30:19,76,40467-61302-43981,976575300749278,GSM,77.410355,23.196945,1000
119,9109709886,7230137725,2020-06-10,19:55:56,80,40478-38004-10762,430484018436777,GSM,77.435074,23.214798,1000


In [295]:
all_lat=[]
all_lon=[]
locs=[]
for (index,row) in f.iterrows():
    locs.append([row['lat'],row['lon']])
for i in range(len(locs)-1):
    all_lat,all_lon=addEdgemap(locs[i],locs[i+1],all_lat,all_lon, 0.7, "end",0.009, 30, 10)


In [296]:
print(all_lat)

[23.184585571289002, 23.268013, None, 23.268013, 23.26162187315502, None, 23.268013, 23.25932971179956, 23.26162187315502, None, 23.268013, 23.231277, None, 23.231277, 23.23880705778951, None, 23.231277, 23.23931094822697, 23.23880705778951, None, 23.231277, 23.196945190430004, None, 23.196945190430004, 23.203590567403655, None, 23.196945190430004, 23.20552422970978, 23.203590567403655, None, 23.196945190430004, 23.214797973632997, None, 23.214797973632997, 23.206586469865446, None, 23.214797973632997, 23.213882582921038, 23.206586469865446, None]


In [297]:
edge_trace=[]
edge_trace.append(go.Scattermapbox(lat=all_lat,lon=all_lon,mode = "lines",marker=go.scattermapbox.Marker(
                size=17,
                color='#EF553B',
                opacity=0.7
            ),
        ))

# edge_trace=go.Scattermapbox(lat=f['lat'],lon=f['lon'],mode = "lines+markers",)
fig=go.Figure(edge_trace,layout={
    'mapbox_style':'open-street-map',
  
    'mapbox':dict(
     
        bearing=0,
        center=dict(
            lat=23.2599,
            lon=77.4126
        ),
        pitch=0,
        zoom=10
    )
    
})
fig.show()

In [323]:
data=[]
selected_numbers=[9109709886,7340496772,7230137725]
colors=['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52']
color_no=0
for number in selected_numbers:
    
    filtered_df=df[df['Caller']==number].sort_values(['Date','Time'])
    print(len(filtered_df))
    all_lat=[]
    all_lon=[]
    locs=[]
    for (index,row) in filtered_df.iterrows():
        locs.append([row['lat'],row['lon']])
    for i in range(len(locs)-1):
        all_lat,all_lon=addEdgemap(locs[i],locs[i+1],all_lat,all_lon, 1, "middle",0.0045, 30, 10)
        data.append(go.Scattermapbox(lat=all_lat,lon=all_lon,hovertext=str(number),mode = "lines",marker=go.scattermapbox.Marker(
                size=17,
                color=colors[color_no],
                opacity=1
            ),
        ))
    color_no+=1
    color_no%=len(colors)
fig=go.Figure(data,layout={
    'mapbox_style':'open-street-map',

    'mapbox':dict(

        bearing=0,
        center=dict(
            lat=23.2599,
            lon=77.4126
        ),
        pitch=0,
        zoom=10
    )

})

10
1
8


In [324]:
fig.show()

In [1]:
start_date="1-6-2020"
start_time="1:30:45"
end_date="20-6-2020"
end_time="21:30:55"

In [2]:
start=start_date+" "+start_time
end=end_date+" "+end_time
def str_time_prop(start, end, format, prop):
    stime = time.mktime(time.strptime(start, format))
    etime = time.mktime(time.strptime(end, format))
    ptime = stime + prop * (etime - stime)
    return time.localtime(ptime)

In [3]:
def random_date(start, end, prop):
    return str_time_prop(start, end, '%d-%m-%Y %H:%M:%S', prop)
def getDate(t):
    return time.strftime("%d-%m-%Y",t)
def getTime(t):
    return time.strftime("%H:%M:%S",t)

In [6]:
import random
import time
t=random_date(start,end,random.random())

In [7]:
getTime(t)

'07:32:16'

In [10]:
pd.read_csv('final_data.csv')

FileNotFoundError: [Errno 2] File final_data.csv does not exist: 'final_data.csv'

In [11]:
df1 = pd.read_csv('data/data.csv')
df2 = pd.read_csv('../data/ipdr_data.csv')

df2['Start Date'] = df2['Start Date'].apply(pd.to_datetime).dt.date

df2.rename(index=str, columns={'IMEI': 'Caller','Start Date':'Date','Start Time':'Time','CELL_ID':'TowerID'}, inplace=True)
df2['Receiver']=20000

frames=[df1,df2]
df=pd.concat(frames)

df=df.fillna(0)

FileNotFoundError: [Errno 2] File data/data.csv does not exist: 'data/data.csv'

In [4]:
import pandas as pd 
df1 = pd.read_csv('../data/data.csv')
df2 = pd.read_csv('../data/ipdr_data.csv')

In [5]:
df2['Start Date'] = df2['Start Date'].apply(pd.to_datetime).dt.date

df2.rename(index=str, columns={'IMEI': 'Caller','Start Date':'Date','Start Time':'Time','CELL_ID':'TowerID'}, inplace=True)
df2['Receiver']=20000

frames=[df1,df2]
df=pd.concat(frames)


In [6]:
df=df.fillna(0)

In [7]:
df

,Caller,Receiver,Date,Time,Duration,TowerID,IMEI,Private IP,Private Port,Public IP,Public Port,Dest IP,DEST PORT,MSISDN,IMSI,Uplink Volume,Downlink Volume,Total Volume,I_RATTYPE
0,8805786023,9616692777,12-06-2020,00:13:12,60,40478-38009-10112,4.496981e+14,0,0.0,0,0.0,0,0,0.000000e+00,0.000000e+00,0.00,0.00,0.00,0
1,8805786023,7703552164,01-06-2020,12:15:54,36,40493-2186-24872,2.759019e+14,0,0.0,0,0.0,0,0,0.000000e+00,0.000000e+00,0.00,0.00,0.00,0
2,8805786023,8751633832,19-06-2020,17:54:26,53,40493-7195-23957,8.971968e+13,0,0.0,0,0.0,0,0,0.000000e+00,0.000000e+00,0.00,0.00,0.00,0
3,8805786023,8650807946,04-06-2020,02:52:33,94,40467-61302-43981,3.980326e+14,0,0.0,0,0.0,0,0,0.000000e+00,0.000000e+00,0.00,0.00,0.00,0
4,8805786023,8763440769,17-06-2020,11:46:46,85,40478-35500-12111,1.886159e+14,0,0.0,0,0.0,0,0,0.000000e+00,0.000000e+00,0.00,0.00,0.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9998315563,20000,2020-06-13,14:06:08,16:23:44,650,0.000000e+00,10.195.70.181,2917.0,141.64.108.51,9234.0,57.35.217.235,58128,2.700330e+11,8.119115e+09,0.07,0.01,0.08,2G
9996,9998315563,20000,2020-06-19,19:20:52,00:42:45,123,0.000000e+00,172.19.207.117,3934.0,70.188.50.160,6188.0,124.105.155.255,40032,9.567068e+11,8.356766e+09,0.07,0.08,0.15,2G
9997,9998315563,20000,2020-06-16,16:22:15,13:07:29,385,0.000000e+00,172.30.99.193,6678.0,27.175.111.249,390.0,86.27.139.245,8003,7.514550e+11,8.553574e+09,0.05,0.06,0.11,3G
9998,9998315563,20000,2020-08-06,01:48:15,05:39:26,424,0.000000e+00,172.31.88.227,3876.0,104.142.156.29,9931.0,164.99.206.122,5222,2.716542e+11,7.321703e+09,0.07,0.05,0.12,3G


In [11]:

import numpy as np
df = pd.read_csv('../data/final_data.csv')
ports_to_apps = {'DEST PORT':'App','0':'nan','5223':'WhatsApp','5228':'WhatsApp', '4244':'WhatsApp', '5222':'WhatsApp', '5242':'WhatsApp','443_':'Skype','443':'SSL',\
    '3478-3481':'Skype','49152-65535':'Skype','80':'Web connection','8080':'Web Connection', \
        '8081': 'Web Connection', '993':'IMAP', '143':'IMAP', '8024':'iTunes', '8027':'iTunes', '8013':'iTunes', \
            '8017':'iTunes', '8003':'iTunes', '7275':'iTunes', '8025':'iTunes', '8009':'iTunes',\
                '58128': 'Xsan', '51637':'Xsan', '61076':'Xsan','40020':'Microsoft Online Games', '40017':'Microsoft Various Games', \
                    '40023':'Microsoft Online Games',\
                    '40019':'Microsoft Online Games', '40001':'Microsoft Online Games', '40004':'Microsoft Online Games', \
                        '40034':'Microsoft Online Games', '40031':'Microsoft Online Games', '40029':'Microsoft Online Games',\
                              '40005':'Microsoft Online Games', '40026': 'Microsoft Online Games', '40008': 'Microsoft Online Games',\
                                  '40032':'Microsoft Online Games'}   # did not take '443':'SSL, Web connection' to avoid double 443
df['App_name'] = df['DEST PORT'].apply(lambda x:ports_to_apps[str(x)] if (ports_to_apps[str(x)]!='nan') else None)
l=df['Receiver'].unique()
l=l[l!=20000] # to remove occurrences of 20000
nodes = np.union1d(df['Caller'].unique(),l )

df['Caller_node'] = df['Caller'].apply(
        lambda x: list(nodes).index(x)) 

data_columns_ipdr = ["Caller","App_name","Private IP","Private Port", "Public IP", "Public Port", "Dest IP","DEST PORT", "MSISDN", "IMSI", "Date","Time",\
    "Duration", "TowerID", "Uplink Volume","Downlink Volume","Total Volume","I_RATTYPE"]
nodeNumber =8805786023

new_df_ipdr = df[(df['Caller_node'] == nodeNumber)][data_columns_ipdr]

In [12]:
new_df_ipdr

,Caller,App_name,Private IP,Private Port,Public IP,Public Port,Dest IP,DEST PORT,MSISDN,IMSI,Date,Time,Duration,TowerID,Uplink Volume,Downlink Volume,Total Volume,I_RATTYPE


In [ ]:
#### Import Libraries #########
import pandas as pd
import base64
import io
import numpy as np
import json
import networkx as nx
import plotly.graph_objects as go
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
from datetime import datetime as dt
from stats import *
import pygraphviz as pgv
import dash_bootstrap_components as dbc
import math
### Import functions for Breadth First Search ###
from addEdge import addEdge

from BFSN import bfs

##### Stylesheet #####
external_stylesheets = [dbc.themes.SANDSTONE]


# Load  Data
df = pd.read_csv('./data/final_data.csv')
#df2 = pd.read_csv('./data/ipdr_data.csv')

#### Create App ###
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.title = 'CDR/IPDR Analyser'
#### Default Variables ####
default_duration_slider_val = [0, 100]
default_time_slider_val = [0,48]
default_caller_receiver_val = 3



# Loop to generate marks for Time
time_str = ['0', '0', ':', '0', '0']
times = {0: {'label': "".join(time_str), "style": {
    "transform": "rotate(-90deg) translateY(-15px)"}}}
for i in range(0, 48):
    if i % 2 == 0:
        time_str[3] = '3'
        times[i+1] = {'label': "".join(time_str), "style": {'display': 'none'}}
    else:
        time_str = list(
            str(int("".join(time_str[0:2])) + 1).zfill(2) + str(':00'))
        times[i+1] = {'label': "".join(time_str),
                      "style": {"transform": "rotate(-90deg) translateY(-15px)"}}
# Generating marks for duration slider
durations = {}
for i in range(0, int(df['Duration'].max()), 5):
    durations[i] = str(i)


coords_to_node = {}  # Dictionary that stores coordinates to node number

node_to_num = {}  # Dictionary that stores node number to phone number
num_to_node = {}  #Dictionary that stores numbers to node
data_columns = ["Caller", "Receiver", "Date",
                "Time", "Duration", "TowerID", "IMEI"]

data_columns_ipdr = ["Caller","App_name","Private IP","Private Port", "Public IP", "Public Port", "Dest IP","DEST PORT", "MSISDN", "IMSI", "Date","Time",\
    "Duration", "TowerID", "Uplink Volume","Downlink Volume","Total Volume","I_RATTYPE"]


ports_to_apps = {'DEST PORT':'App','0':'nan','5223':'WhatsApp','5228':'WhatsApp', '4244':'WhatsApp', '5222':'WhatsApp', '5242':'WhatsApp','443_':'Skype','443':'SSL',\
    '3478-3481':'Skype','49152-65535':'Skype','80':'Web connection','8080':'Web Connection', \
        '8081': 'Web Connection', '993':'IMAP', '143':'IMAP', '8024':'iTunes', '8027':'iTunes', '8013':'iTunes', \
            '8017':'iTunes', '8003':'iTunes', '7275':'iTunes', '8025':'iTunes', '8009':'iTunes',\
                '58128': 'Xsan', '51637':'Xsan', '61076':'Xsan','40020':'Microsoft Online Games', '40017':'Microsoft Various Games', \
                    '40023':'Microsoft Online Games',\
                    '40019':'Microsoft Online Games', '40001':'Microsoft Online Games', '40004':'Microsoft Online Games', \
                        '40034':'Microsoft Online Games', '40031':'Microsoft Online Games', '40029':'Microsoft Online Games',\
                              '40005':'Microsoft Online Games', '40026': 'Microsoft Online Games', '40008': 'Microsoft Online Games',\
                                  '40032':'Microsoft Online Games'}   # did not take '443':'SSL, Web connection' to avoid double 443


# Color scale of edges
viridis = cm.get_cmap('viridis', 12)
def preprocess_data(df):
    # nodes
    l=df['Receiver'].unique()
    l=l[l!=20000] # to remove occurrences of 20000
    nodes = np.union1d(df['Caller'].unique(),l )
    # Define Color
    df['Dura_color'] = (df['Duration']/df['Duration'].max()).apply(viridis)
    df['Date'] = df['Date'].apply(pd.to_datetime).dt.date


    df['Caller_node'] = df['Caller'].apply(
        lambda x: list(nodes).index(x))  # Caller Nodes
    
#    df['Receiver_node'] = df['Receiver'].apply(lambda x: list(nodes).index(x))
    df['Receiver_node'] = df['Receiver'].apply(lambda x: list(nodes).index(x) if x!=20000 else -1)

    df['IMEI_node'] = df['Caller'].apply(lambda x: list(nodes).index(x) if x!=20000 else -1)
    df['App_name'] = df['DEST PORT'].apply(lambda x:ports_to_apps[str(x)] if (ports_to_apps[str(x)]!='nan') else None)

    coords_to_node.clear()
    num_to_node.clear()
    node_to_num.clear()

preprocess_data(df)
#### Plots ####
# Plot Graph of calls

fig = go.Figure()
pos = {}

def plot_network(df):
    G = nx.DiGraph()  # networkX Graph
    # Reciever Nodes
    df=df[df['Receiver_node']!=-1]
    def make_graph(x):
            G.add_edge(x["Caller_node"], x["Receiver_node"])

    df.apply(make_graph, axis=1)  # Make a graph
    pos = nx.nx_agraph.graphviz_layout(G)  # Position of Points

    # Add Edges to Plot
    edge_trace = []

    def add_coords(x):
        x0, y0 = pos[x['Caller_node']]
        x1, y1 = pos[x['Receiver_node']]

        node_to_num[x['Caller_node']] = x['Caller']
        node_to_num[x['Receiver_node']] = x['Receiver']
        num_to_node[x['Caller']] = x['Caller_node']
        num_to_node[x['Receiver']] = x['Receiver_node']
        edges_x,edges_y=addEdge((x0,y0),(x1,y1),[],[])
        edge_trace.append(dict(type='scatter',
                               x=edges_x, y=edges_y,
                               showlegend=False,
                               line=dict(
                                   width=2, color='rgba'+str(x['Dura_color']).replace(']', ')').replace('[', '(')),
                               hoverinfo='none',
                               mode='lines',
        ))  # Graph object for each connection
       
    df.apply(add_coords, axis=1)  # Adding edges

    # adding points
    node_x = []
    node_y = []
    total_duration = []
    hover_list = []
    for node in pos:
        x, y = pos[node]
        coords_to_node[(x, y)] = node
        hover_list.append(str(node_to_num[node]))
        node_x.append(x)
        node_y.append(y)
        total_duration.append(27*pow((df[(df['Caller_node']==node)|(df['Receiver_node']==node)]['Duration'].sum())/df['Duration'].max(),0.3))
    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers',
        hovertext = hover_list,
        hoverinfo='text',
        showlegend=False,
        marker=dict(
            size=total_duration,
            showscale=True,
            line_width=2,
            line_color='black'))  # Object for point scatter plot
    fig = go.Figure(data=edge_trace+[node_trace],
                    layout=go.Layout(
                   
                    titlefont_size=16,
                 

                    hovermode='closest',
                    margin=dict(b=20, l=5, r=5, t=40),
                    annotations=[dict(
                        showarrow=True,
                        xref="paper", yref="paper",
                        x=0.005, y=-0.002)],
                    xaxis=dict(showgrid=False, zeroline=False,
                               showticklabels=False),
                    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                    )  # Complete Figure
    fig.update_layout(transition_duration=500)  # Transition
    #fig.update_layout(hover_label_align = 'right')
    fig.update_layout(
         hoverlabel = dict(bgcolor='white',font_size = 15,font_family = 'Rockwell')
    )
    # fig.update_layout(clickmode='event+select')  # Event method
    fig.update_layout(yaxis = dict(scaleanchor = "x", scaleratio = 1), plot_bgcolor='rgb(255,255,255)')
    #fig.update_traces(marker_size=20)  # marker size
    return fig

# Create an image for the same. -- What?


##### Layout of App #####
app.layout = html.Div(children=[

    #Child 1
    html.Div(children=[
        html.H1(children='CDR Analyser'),  # Title
        html.H3(children='''
        Analyse the phone calls between people.
    '''),  # Subtitle
        html.Div(
            id='date-selected'
        ),  # Date Selected Indicator
        html.H4(id='message'),  # Message
    ],id='header-text'),

    #Child 2
    dbc.Row(children=[

        #Child 2.1
        dbc.Col(children=[
            #2.1.1
            html.H3(
                'Filters'
            ),
            #2.1.2
            html.H5(
                'From:'
            ),
            #2.1.3
            dcc.DatePickerSingle(
                id='date-picker1',
                min_date_allowed=df['Date'].min(),
                max_date_allowed=df['Date'].max(),
                initial_visible_month=dt(2020, 6, 5),
                date=str(dt(2020, 6, 17, 0, 0, 0))
            ),  # Data Picker
            html.H5(
                'To:'
            ),
            dcc.DatePickerSingle(
                id='date-picker2',
                min_date_allowed=df['Date'].min(),
                max_date_allowed=df['Date'].max(),
                initial_visible_month=dt(2020, 6, 5),
                date=str(dt(2020, 6, 17, 0, 0, 0))
            ),
            #2.1.6
            dcc.RangeSlider(
                id='duration-slider',
                min=0,
                max=df['Duration'].max(),
                step=None,
                marks=durations,

                value=default_duration_slider_val,
                dots=True,

            ),  # Duration Slider

            dcc.RangeSlider(
                id='time-slider',
                min=0,
                max=48,
                step=None,
                marks=times,
                dots=True,
                value=default_time_slider_val,
                pushable=1

            ),  # Time Slider
            html.H5(
                'Condition for Caller/Reciever'
            ),
            #2.1.9
            dcc.Dropdown(
                id='select-caller-receiver',
                options=[{'label': 'Only Caller', 'value': 1}]+[{'label': 'Only Receiver', 'value': 2}]+[
                    {'label': 'Either Caller or Reciever', 'value': 3}]+[{'label': 'Both Caller and Reciever', 'value': 4}],
                value=default_caller_receiver_val,
            ),  # Select if you want the select the numbers to be from Caller/Reciever/Both/Either
           
            html.H5(
                'Select Caller:'
            ),
            dcc.Dropdown(
                id='caller-dropdown',
                options=[{'label': 'None', 'value': 'None'}] + \
                [{'label': k, 'value': k} for k in df['Caller'].unique()],
                value='None',
                multi=True,
            ),  # Dropdown for Caller
            html.H5(
                'Select Reciever:'
            ),
            #2.1.13
            dcc.Dropdown(
                id='receiver-dropdown',
                options=[{'label': 'None', 'value': 'None'}] + \
                [{'label': k, 'value': k} for k in df['Receiver'].unique()],
                value='None',
                multi=True,
            ),  # Dropdown for Reciever,
            html.H5(
                'Find:'
            ),
            dcc.Dropdown(
                id='find-dropdown',
                options=[{'label': 'None', 'value': 'None'}] + \
                [{'label': k, 'value': k} for k in pd.unique(df[['Caller','Receiver']].values.ravel())],
                value='None',
                multi=True,
            ),   # Dropdown for finding,
            #2.1.16
            html.Div([
                dcc.Upload(
                    id='upload-data',
                    children=html.Div([
                        'Drag and Drop or ',
                        html.A('Select a File')
                    ]),
                    style={
                        'width': '100%',
                        'height': '60px',
                        'lineHeight': '60px',
                        'borderWidth': '1px',
                        'borderStyle': 'dashed',
                        'borderRadius': '5px',
                        'textAlign': 'center',
                        'margin-bottom':'12px',
                        'margin-top':'12px'
                    },
                    # Allow multiple files to be uploaded
                    multiple=False
                ),
                html.Div(id='output-data-upload'),
            ]),

            html.Button('Reset Filters', id='reset-button', n_clicks=0)
            ],
            id='filters',lg=3),  # Filters  #Child 2.1 done

        dbc.Col(
           [ html.H3('Network Graph : '),


            dcc.Graph(
                id='network-plot'

            ),

            html.H5('The size of the dots denote the total duration of the caller/callee'),

         	dcc.Markdown("""
             		1. The Duration the selected person has spent on exchanging calls over time
            		"""),
         	dcc.Graph(id='duration-plot'),


            ],  id='plot-area',lg=6


            ),     # Network Plot




        dbc.Col(children=[
            html.H3('Statistics'),
            html.Div([
                dcc.Markdown("""
                **Hover To Get Stats** \n
                Mouse over nodes in the graph to get statistics.
            """),
                html.Pre(id='hover-data',)
            ],),  # Hover Data Container

            html.Div([
                
                dcc.Markdown("""
                **Click to get CDR for a number** \n
                Click on points in the graph to get the call data records.
            """),
                html.Pre(id='click-data', ),
                dcc.Graph(
                    id='pie-chart'
                ),
                dcc.Markdown("""
                **Click to get IPDR for a number** \n
                Click on points in the graph to get the IPDR for the number.
            """),
                html.Pre(id='click-data-ipdr', )

            ], ),  # Click Data Container


            html.Div([
                dcc.Markdown("""
                **Select to see connected people** \n
                Select using rectangle/lasso or by using your mouse.(Use Shift for multiple selections)
            """),
                html.Pre(id='selected-data', ),
            ], )  # Selection Data Container


        ],id='stats',lg=3)

    ],className='container-mid'),
    html.Div(
        id='filtered-data',
        style={'display': 'none'}
    ),
    # Filtered Data
])

#### Callbacks ####
# Callback to update df used for plotting
# Callback to filter dataframe
# REMEMBER WHILE EDITING (RWI): THIS IS A TWO OUTPUT FUNCTION


@app.callback(
    [Output(component_id='filtered-data', component_property='children'),
     Output(component_id='message', component_property='children')],
    [Input('upload-data', 'contents'),Input(component_id='date-picker1', component_property='date'),Input(component_id='date-picker2', component_property='date'), Input(component_id='duration-slider', component_property='value'), Input(component_id='time-slider', component_property='value'),
     Input(component_id='select-caller-receiver', component_property='value'), Input(component_id='caller-dropdown', component_property='value'), Input(component_id='receiver-dropdown', component_property='value')]
)
def update_filtered_div_caller(contents, selected_date1, selected_date2, selected_duration, selected_time, selected_option, selected_caller, selected_receiver):
    # Date,Time,Duration Filter
    if contents is not None:
        content_type, content_string = contents.split(',')
        decoded = base64.b64decode(content_string)
        global df
        df = pd.read_csv(io.StringIO(decoded.decode('utf-8')))
        preprocess_data(df)

    filtered_df = df[(df['Date'] >= pd.to_datetime(selected_date1)) & (df['Date'] <= pd.to_datetime(selected_date2))
                     & ((df['Duration'] >= selected_duration[0]) & (df['Duration'] <= selected_duration[1]))
                     & ((df['Time'] < times[selected_time[1]]['label']) & (df['Time'] >= times[selected_time[0]]['label']))].reset_index(drop=True)

    # Number Filter
    # If Caller is Selected
    if(selected_option == 1):
        if selected_caller != 'None':
            filtered_df = filtered_df[(filtered_df['Caller'].isin(
                list(selected_caller)))].reset_index(drop=True)

   # If Receiver is selected
    if(selected_option == 2):
        if selected_receiver != 'None':
            filtered_df = filtered_df[(filtered_df['Receiver'].isin(
                (selected_receiver)))].reset_index(drop=True)
   # If the option either is selected
    if(selected_option == 3):
        if selected_caller != 'None' or selected_receiver != 'None':
            filtered_df = filtered_df[((filtered_df['Caller'].isin(list(selected_caller))) | (
                filtered_df['Receiver'].isin(list(selected_receiver))))].reset_index(drop=True)
    # If option both is selected
    if(selected_option == 4):
        if selected_caller != 'None' and selected_receiver != 'None':
            filtered_df = df[((filtered_df['Caller'].isin(list(selected_caller))) & (
                filtered_df['Receiver'].isin(list(selected_receiver))))].reset_index(drop=True)
    if filtered_df.shape[0] == 0:
        # No update since nothing matches
        return dash.no_update, 'Nothing Matches that Query'
    else:
        # Update Filtered Dataframe
        return filtered_df.to_json(date_format='iso', orient='split'), 'Updated'
# Callback for hover data
# Display Stats in hoverdata


@app.callback(
    Output('hover-data', 'children'),
    [Input('network-plot', 'hoverData'), Input(component_id='filtered-data', component_property='children')])
def display_hover_data(hoverData, filtered_data):

    df = pd.read_json(filtered_data, orient='split')
    if hoverData is not None:
        # Get node number corresponding to the point.

        nodeNumber = coords_to_node[(
            hoverData['points'][0]['x'], hoverData['points'][0]['y'])]
        hd = 'Selected Number: ' + \
            str(node_to_num[nodeNumber]) + '\n'  # hd: Hover Data string

        # Functions are from stats.py
        hd += "Mean Duration : " + str(meanDur(nodeNumber, df)) + "\n"
        hd += "Peak Hours(duration):\n"
        m = peakHours(nodeNumber, df)
        for x in m:
            hd += "\t\t  " + str(x)+"-"+str(x+1)+" : "+str(m[x])+"\n"
        z = ogIc(nodeNumber, df)  # Outgoing Incoming
        hd += "No. of Outgoing Calls: " + str(z[0])+"\n"
        hd += "No. of Incomming Calls: " + str(z[1])+"\n"
        z = mostCalls(nodeNumber, df)  # Most Calls
        hd += "Most Calls to: " + str(z[0]) + "\n"
        hd += "Most Calls from: " + str(z[1]) + "\n"
        hd += "Most Calls: " + str(z[2]) + "\n"
        return hd
    return "Hover data..."

# Callback for Network Plot
# Show table for the clicked phone number


@app.callback(
    [Output('click-data', 'children'), Output('pie-chart','figure'),Output('click-data-ipdr', 'children'), Output('duration-plot','figure')], #Suggest to put all extra plots in this callback's output...
    [Input('network-plot', 'clickData'),Input(component_id='filtered-data', component_property='children')])
def display_click_data(clickData,filtered_data):
    df = pd.read_json(filtered_data, orient='split')
    if clickData is not None:
        nodeNumber = coords_to_node[(
            clickData['points'][0]['x'], clickData['points'][0]['y'])]
        groups=df[df['IMEI_node']==nodeNumber].groupby('App_name')['Caller'].count()
        print(groups,nodeNumber)
        fig = go.Figure(data=dict(type='pie',values=groups,labels=groups.index))
        fig.update_layout(showlegend=False)
        #print(fig)
        # Filtering DF
        new_df = df[((df['Receiver_node'] !=-1)&(df['Caller_node'] == nodeNumber) | (df['Receiver_node'] == nodeNumber))][data_columns]

        new_df_ipdr = df[(df['Caller_node'] == nodeNumber)][data_columns_ipdr]
        print(new_df_ipdr['App_name'].unique())
        new_df_ipdr=new_df_ipdr[new_df_ipdr['App_name'].notna()]

        return new_df.to_string(index=False), fig,new_df_ipdr.to_string(index=False), plot_Duration(new_df)
    return "Click on a node to view more data",go.Figure(),"Click on a node to view more data", go.Figure() #DO NOT RETURN HERE 'None', otherwise duration-plot will always be empty.



#Callback to output the new figure for Duration plot of selected node.
def plot_Duration(new_df):

	if new_df is not None:

		x = sorted(new_df["Date"])
		y = new_df["Duration"]
		fig = go.Figure([ go.Scatter(x = x, y = y,
	                           mode='lines+markers',
	                            name='plot') ])  
		return fig
	else:
		return None



# Callback for Selected Data
@app.callback(
    Output('selected-data', 'children'),
    [Input('network-plot', 'selectedData'), Input(component_id='filtered-data', component_property='children')])
def display_selected_data(selectedData, filtered_data):
    df = pd.read_json(filtered_data, orient='split')
    # TODO #3 Graph should also be filtered and only nodes in component should be displayed
    if selectedData is not None:
        l = []
        for point in selectedData['points']:
            l.append(node_to_num[coords_to_node[point['x'], point['y']]])
        components = bfs(l, df)
        s = ""
        i = 1
        for component in components:
            if not component:
                continue
            s += "Component "+str(i)+":\n"
            i += 1
            for number in component:
                s += "\t" + str(number) + "\n"
        return s
    return json.dumps(selectedData, indent=2)


# Callback to update network plot
@app.callback(
    Output(component_id='network-plot', component_property='figure'),
    [Input(component_id='filtered-data', component_property='children')]
)
def update_network_plot_caller(filtered_data):
    return plot_network(pd.read_json(filtered_data, orient='split'))

# TODO UPDATE THESE CALLBACKS TO INCLUDE TIME AND DURATION UPDATES
# Callback to change selectors including caller no. according to date


@app.callback(
    Output(component_id='caller-dropdown', component_property='options'),
    [Input(component_id='date-picker1', component_property='date'),Input(component_id='date-picker2', component_property='date')]
)
def update_phone_div_caller(selected_date1, selected_date2):
    return [{'label': 'None', 'value': ''}]+[{'label': k, 'value': k} for k in df[(df['Date'] >= pd.to_datetime(selected_date1)) & (df['Date']<=pd.to_datetime(selected_date2))]['Caller'].unique()]

# Callback to change reciever according  to dates


@app.callback(
    Output(component_id='receiver-dropdown', component_property='options'),
    [Input(component_id='date-picker1', component_property='date'),Input(component_id='date-picker2', component_property='date')]
)
def update_phone_div_receiver1(selected_date1, selected_date2):
    return [{'label': 'None', 'value': ''}]+[{'label': k, 'value': k} for k in df[(df['Date'] >= pd.to_datetime(selected_date1)) & (df['Date']<=pd.to_datetime(selected_date2))]['Receiver'].unique()]


@app.callback(
    Output(component_id='find-dropdown', component_property='options'),
    [Input(component_id='date-picker1', component_property='date'),Input(component_id='date-picker2', component_property='date')]
)
def update_phone_div_receiver2(selected_date1, selected_date2):
    return [{'label': 'None', 'value': ''}]+[{'label': k, 'value': k} for k in pd.unique(df[(df['Date'] >= pd.to_datetime(selected_date1)) & (df['Date']<=pd.to_datetime(selected_date2))][['Receiver','Caller']].values.ravel())]


# Reset Filters
@app.callback(
	[Output('date-picker1', 'date'), Output('date-picker2', 'date'), Output('duration-slider', 'value'), Output('time-slider', 'value'), Output('select-caller-receiver', 'value'), Output('caller-dropdown', 'value'), Output('receiver-dropdown', 'value'), Output('find-dropdown', 'value')],
	[Input('reset-button', 'n_clicks')]
	)
def ResetFilters(button_reset):
	return str(dt(2020, 6, 17, 0, 0, 0)), str(dt(2020, 6, 17, 0, 0, 0)), default_duration_slider_val, default_time_slider_val, default_caller_receiver_val, 'None', 'None', 'None'


#### Run Server ####
if __name__ == '__main__':
    app.run_server(debug=True, host='0.0.0.0')